In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, TimeDistributed, Flatten, BatchNormalization
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# 데이터셋 불러오기
file_path = 'C:\\Users\\co279\\mp1.csv'
data = pd.read_csv(file_path)

# 필요 없는 컬럼 제거
data = data.drop(columns=['last_name, first_name'])

# 결측치 처리 (예: 0으로 대체)
data = data.fillna(0)

# 2020, 2021, 2022년에 모두 존재하는 player_id 추출
data_2016 = data[data['year'] == 2016]
data_2017 = data[data['year'] == 2017]
data_2018 = data[data['year'] == 2018]
data_2019 = data[data['year'] == 2019]

player_ids_2016 = set(data_2016['player_id'].unique())
player_ids_2017 = set(data_2017['player_id'].unique())
player_ids_2018 = set(data_2018['player_id'].unique())
player_ids_2019 = set(data_2019['player_id'].unique())

common_player_ids = player_ids_2016 & player_ids_2017 & player_ids_2018 & player_ids_2019

# 공통 player_id에 해당하는 데이터 추출
common_data = data[data['player_id'].isin(common_player_ids)]

# 2020, 2021, 2022년에 해당하는 데이터만 추출
final = common_data[common_data['year'].isin([2016, 2017, 2018])]
final = final.sort_values(by=['player_id', 'year'])

# 필요한 컬럼 선택 (year 제외)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# 독립변수와 종속변수 분리
X = final[features].values
y = final[target].values

# 데이터 스케일링
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# 시계열 데이터 형태로 변환
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 3  # 시퀀스 길이 설정
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# 학습 데이터와 전체 데이터를 동일하게 설정
X_train, y_train = X_seq, y_seq

In [2]:
# LSTM_A

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_A
    model_A = Sequential()
    model_A.add(LSTM(128, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_A.add(LSTM(128, return_sequences=True))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Flatten())
    model_A.add(Dense(1024, activation="relu"))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_A.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_A = model_A.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_A = model_A.predict(X_2019_seq)

    # 스케일 복원
    y_pred_A = scaler_y.inverse_transform(y_pred_scaled_A)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_A = np.sqrt(mean_squared_error(y_test_actual, y_pred_A))
    rmse_list.append(rmse_A)

    # MAE 계산
    mae_A = mean_absolute_error(y_test_actual, y_pred_A)
    mae_list.append(mae_A)
    
    # MAPE 계산
    mape_A = mean_absolute_percentage_error(y_test_actual, y_pred_A)
    mape_list.append(mape_A)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_A}, MAE: {mae_A}, MAPE: {mape_A}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 17s 531ms/step - loss: 0.0760 - accuracy: 0.0000e+00 - val_loss: 0.0393 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 34ms/step - loss: 0.0317 - accuracy: 0.0025 - val_loss: 0.0292 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 38ms/step - loss: 0.0254 - accuracy: 0.0025 - val_loss: 0.0342 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 33ms/step - loss: 0.0228 - accuracy: 0.0025 - val_loss: 0.0286 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 37ms/step - loss: 0.0210 - accuracy: 0.0025 - val_loss: 0.0292 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 35ms/step - loss: 0.0197 - accuracy: 0.0025 - val_loss: 0.0279 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 36ms/step - loss: 0.0182 - accuracy: 0.0025 - val_loss: 0.0266 - val_accurac

In [3]:
# LSTM_B

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_B
    model_B = Sequential()
    model_B.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_B.add(LSTM(16, return_sequences=True))
    model_B.add(LSTM(8, return_sequences=True))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Flatten())
    model_B.add(Dense(512, activation="relu"))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Dense(64, activation="relu"))
    model_B.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_B.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_B = model_B.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_B = model_B.predict(X_2019_seq)

    # 스케일 복원
    y_pred_B = scaler_y.inverse_transform(y_pred_scaled_B)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_B = np.sqrt(mean_squared_error(y_test_actual, y_pred_B))
    rmse_list.append(rmse_B)

    # MAE 계산
    mae_B = mean_absolute_error(y_test_actual, y_pred_B)
    mae_list.append(mae_B)
    
    # MAPE 계산
    mape_B = mean_absolute_percentage_error(y_test_actual, y_pred_B)
    mape_list.append(mape_B)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_B}, MAE: {mae_B}, MAPE: {mape_B}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 20s 609ms/step - loss: 0.1226 - accuracy: 0.0025 - val_loss: 0.0427 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 28ms/step - loss: 0.0374 - accuracy: 0.0025 - val_loss: 0.0290 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 25ms/step - loss: 0.0243 - accuracy: 0.0025 - val_loss: 0.0454 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 23ms/step - loss: 0.0267 - accuracy: 0.0025 - val_loss: 0.0341 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 24ms/step - loss: 0.0230 - accuracy: 0.0025 - val_loss: 0.0282 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 25ms/step - loss: 0.0215 - accuracy: 0.0025 - val_loss: 0.0319 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 33ms/step - loss: 0.0214 - accuracy: 0.0025 - val_loss: 0.0299 - val_accuracy: 0

In [4]:
# LSTM_C

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_C
    model_C = Sequential()
    model_C.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_C.add(LSTM(32, return_sequences=True))
    model_C.add(BatchNormalization())
    model_C.add(Flatten())
    model_C.add(Dense(512, activation="relu"))
    model_C.add(BatchNormalization())
    model_C.add(Dense(64, activation="relu"))
    model_C.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_C.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_C = model_C.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_C = model_C.predict(X_2019_seq)

    # 스케일 복원
    y_pred_C = scaler_y.inverse_transform(y_pred_scaled_C)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_C = np.sqrt(mean_squared_error(y_test_actual, y_pred_C))
    rmse_list.append(rmse_C)

    # MAE 계산
    mae_C = mean_absolute_error(y_test_actual, y_pred_C)
    mae_list.append(mae_C)
    
    # MAPE 계산
    mape_C = mean_absolute_percentage_error(y_test_actual, y_pred_C)
    mape_list.append(mape_C)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_C}, MAE: {mae_C}, MAPE: {mape_C}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 24s 573ms/step - loss: 0.1978 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 22ms/step - loss: 0.2014 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 21ms/step - loss: 0.1919 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 22ms/step - loss: 0.1953 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 21ms/step - loss: 0.1934 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 20ms/step - loss: 0.1900 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 24ms/step - loss: 0.1902 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0

In [5]:
# LSTM_D

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_D
    model_D = Sequential()
    model_D.add(Masking(mask_value=0.0, input_shape=(seq_length, X_train.shape[2])))
    model_D.add(LSTM(64, return_sequences=True))
    model_D.add(LSTM(64, return_sequences=True))
    model_D.add(Dropout(rate=0.5))
    model_D.add(TimeDistributed(Dense(10)))
    model_D.add(Flatten())
    model_D.add(Dense(512, activation="relu"))
    model_D.add(Dropout(rate=0.5))
    model_D.add(Dense(64, activation="relu"))
    model_D.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_D.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_D = model_D.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_D = model_D.predict(X_2019_seq)

    # 스케일 복원
    y_pred_D = scaler_y.inverse_transform(y_pred_scaled_D)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_D = np.sqrt(mean_squared_error(y_test_actual, y_pred_D))
    rmse_list.append(rmse_D)

    # MAE 계산
    mae_D = mean_absolute_error(y_test_actual, y_pred_D)
    mae_list.append(mae_D)
    
    # MAPE 계산
    mape_D = mean_absolute_percentage_error(y_test_actual, y_pred_D)
    mape_list.append(mape_D)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_D}, MAE: {mae_D}, MAPE: {mape_D}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 38s 755ms/step - loss: 0.1496 - accuracy: 0.0025 - val_loss: 0.0457 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 24ms/step - loss: 0.0413 - accuracy: 0.0025 - val_loss: 0.0308 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0287 - accuracy: 0.0025 - val_loss: 0.0481 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0270 - accuracy: 0.0025 - val_loss: 0.0291 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0249 - accuracy: 0.0025 - val_loss: 0.0341 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0255 - accuracy: 0.0025 - val_loss: 0.0323 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0213 - accuracy: 0.0025 - val_loss: 0.0280 - val_accuracy: 0

In [6]:
# LSTM_E

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_E
    model_E = Sequential()
    model_E.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_E.add(LSTM(32, return_sequences=True))
    model_E.add(Dropout(rate=0.5))
    model_E.add(Flatten())
    model_E.add(Dense(512, activation="relu"))
    model_E.add(Dropout(rate=0.5))
    model_E.add(Dense(64, activation="relu"))
    model_E.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_E.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_E = model_E.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_E = model_E.predict(X_2019_seq)

    # 스케일 복원
    y_pred_E = scaler_y.inverse_transform(y_pred_scaled_E)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_E = np.sqrt(mean_squared_error(y_test_actual, y_pred_E))
    rmse_list.append(rmse_E)

    # MAE 계산
    mae_E = mean_absolute_error(y_test_actual, y_pred_E)
    mae_list.append(mae_E)
    
    # MAPE 계산
    mape_E = mean_absolute_percentage_error(y_test_actual, y_pred_E)
    mape_list.append(mape_E)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_E}, MAE: {mae_E}, MAPE: {mape_E}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 22s 349ms/step - loss: 0.1115 - accuracy: 0.0025 - val_loss: 0.0303 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0450 - accuracy: 0.0000e+00 - val_loss: 0.0410 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0284 - accuracy: 0.0025 - val_loss: 0.0499 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0248 - accuracy: 0.0025 - val_loss: 0.0301 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 23ms/step - loss: 0.0252 - accuracy: 0.0025 - val_loss: 0.0378 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0228 - accuracy: 0.0025 - val_loss: 0.0388 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0222 - accuracy: 0.0025 - val_loss: 0.0311 - val_accurac

In [7]:
y_pred_A_list = [item[0] for item in y_pred_A]

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

df = pd.DataFrame({'Actual': data_19[target], 'Predicted': y_pred_A_list})
df['diff'] = df['Predicted'] - df['Actual']
print(df.sort_values(by='Actual'))

      Actual  Predicted      diff
3247    1.71   2.774909  1.064909
3275    2.25   2.936654  0.686654
3165    2.46   2.939773  0.479773
3192    2.63   2.267740 -0.362260
3336    2.75   2.701624 -0.048376
3196    2.79   3.107215  0.317215
3272    3.05   3.102515  0.052515
3207    3.09   2.478171 -0.611829
3166    3.14   3.836476  0.696476
3235    3.16   3.277169  0.117169
3295    3.22   3.450481  0.230481
3153    3.22   3.779175  0.559175
3334    3.27   3.212042 -0.057958
3167    3.28   2.884662 -0.395338
3248    3.30   3.581317  0.281317
3309    3.50   2.808822 -0.691178
3288    3.57   3.339499 -0.230501
3169    3.57   4.252724  0.682724
3182    3.58   2.760346 -0.819654
3154    3.63   3.907868  0.277868
3312    3.65   3.682236  0.032236
3198    3.74   4.055720  0.315720
3266    3.77   4.234779  0.464779
3159    3.77   3.483694 -0.286306
3230    3.83   3.313064 -0.516936
3243    3.86   4.382129  0.522129
3221    3.86   4.027235  0.167235
3231    3.95   3.704851 -0.245149
3185    4.12  